In [ ]:
# run a regression over the first 3 years (Ferguson) and find significant factors
# use elimination methods to find best factors
# Then look at what the other coaches did that messed up
# maybe do a prediction to find results in short term

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
tables = []
for i in os.listdir('./english-premier-league_zip/archive/'):
     f = os.path.join('./english-premier-league_zip/archive/', i)
     print(f)
     df = pd.read_csv(f)
     tables.append(df)
df1 = pd.concat(tables)
df1

./english-premier-league_zip/archive/season-1011.csv
./english-premier-league_zip/archive/season-1112.csv
./english-premier-league_zip/archive/season-1213.csv
./english-premier-league_zip/archive/season-1415.csv
./english-premier-league_zip/archive/season-1516.csv
./english-premier-league_zip/archive/season-1617.csv


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSH,PSD,PSA,PSCH,PSCD,PSCA
0,E0,14/08/10,Aston Villa,West Ham,3.0,0.0,H,2.0,0.0,H,...,1.45,1.40,3.00,2.78,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,14/08/10,Blackburn,Everton,1.0,0.0,H,1.0,0.0,H,...,2.20,2.05,1.85,1.75,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,14/08/10,Bolton,Fulham,0.0,0.0,D,0.0,0.0,D,...,1.60,1.55,2.56,2.36,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,14/08/10,Chelsea,West Brom,6.0,0.0,H,2.0,0.0,H,...,2.06,2.01,1.91,1.85,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,14/08/10,Sunderland,Birmingham,2.0,2.0,D,1.0,0.0,H,...,1.64,1.55,2.50,2.38,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,21/05/17,Liverpool,Middlesbrough,3.0,0.0,H,1.0,0.0,H,...,2.02,1.97,1.95,1.90,1.14,9.31,27.82,1.15,10.50,19.09
376,E0,21/05/17,Man United,Crystal Palace,2.0,0.0,H,2.0,0.0,H,...,2.19,2.11,1.85,1.79,2.44,3.40,3.14,2.35,3.38,3.35
377,E0,21/05/17,Southampton,Stoke,0.0,1.0,A,0.0,0.0,D,...,2.03,1.98,1.93,1.88,1.80,3.95,4.73,1.64,4.37,5.53
378,E0,21/05/17,Swansea,West Brom,2.0,1.0,H,0.0,1.0,A,...,2.11,2.06,1.86,1.82,2.10,3.68,3.67,2.29,3.45,3.40


In [3]:
drop_columns = 'Div|HC|AC|HR|AR|HY|AY|Referee'
df1 = df1.iloc[:, 0:23]
df1 = df1.loc[:,~df1.columns.str.contains(drop_columns)]

df1.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF
0,14/08/10,Aston Villa,West Ham,3.0,0.0,H,2.0,0.0,H,23.0,12.0,11.0,2.0,15.0,15.0
1,14/08/10,Blackburn,Everton,1.0,0.0,H,1.0,0.0,H,7.0,17.0,2.0,12.0,19.0,14.0
2,14/08/10,Bolton,Fulham,0.0,0.0,D,0.0,0.0,D,13.0,12.0,9.0,7.0,12.0,13.0
3,14/08/10,Chelsea,West Brom,6.0,0.0,H,2.0,0.0,H,18.0,10.0,13.0,4.0,10.0,10.0
4,14/08/10,Sunderland,Birmingham,2.0,2.0,D,1.0,0.0,H,6.0,13.0,2.0,7.0,13.0,10.0


In [4]:
df1 = df1.rename(columns={'Date':'Game_Date','HomeTeam':'Home_Team', 'AwayTeam':'Away_Team', 'FTHG':'Full_Time_Home_Team_Goals', 'FTAG':'Full_Time_Away_Team_Goals',
                          'FTR':'Full_Time_Result', 'HTHG':'Half_Time_Home_Team_Goals', 'HTAG':'Half_Time_Away_Team_Goals', 'HTR':'Half_Time_Result',
                          'HS':'Home_Team_Shots','AS':'Away_Team_Shots', 'HST':'Home_Team_Shots_on_Target', 'AST': 'Away_Team_Shots_on_Target',
                          'HF':'Home_Fouls', 'AF':'Away_Fouls'})

df1.head()

,Game_Date,Home_Team,Away_Team,Full_Time_Home_Team_Goals,Full_Time_Away_Team_Goals,Full_Time_Result,Half_Time_Home_Team_Goals,Half_Time_Away_Team_Goals,Half_Time_Result,Home_Team_Shots,Away_Team_Shots,Home_Team_Shots_on_Target,Away_Team_Shots_on_Target,Home_Fouls,Away_Fouls
0,14/08/10,Aston Villa,West Ham,3.0,0.0,H,2.0,0.0,H,23.0,12.0,11.0,2.0,15.0,15.0
1,14/08/10,Blackburn,Everton,1.0,0.0,H,1.0,0.0,H,7.0,17.0,2.0,12.0,19.0,14.0
2,14/08/10,Bolton,Fulham,0.0,0.0,D,0.0,0.0,D,13.0,12.0,9.0,7.0,12.0,13.0
3,14/08/10,Chelsea,West Brom,6.0,0.0,H,2.0,0.0,H,18.0,10.0,13.0,4.0,10.0,10.0
4,14/08/10,Sunderland,Birmingham,2.0,2.0,D,1.0,0.0,H,6.0,13.0,2.0,7.0,13.0,10.0


In [5]:
df1['Game_Date'] = pd.to_datetime(df1['Game_Date'])
df1.head()

,Game_Date,Home_Team,Away_Team,Full_Time_Home_Team_Goals,Full_Time_Away_Team_Goals,Full_Time_Result,Half_Time_Home_Team_Goals,Half_Time_Away_Team_Goals,Half_Time_Result,Home_Team_Shots,Away_Team_Shots,Home_Team_Shots_on_Target,Away_Team_Shots_on_Target,Home_Fouls,Away_Fouls
0,2010-08-14,Aston Villa,West Ham,3.0,0.0,H,2.0,0.0,H,23.0,12.0,11.0,2.0,15.0,15.0
1,2010-08-14,Blackburn,Everton,1.0,0.0,H,1.0,0.0,H,7.0,17.0,2.0,12.0,19.0,14.0
2,2010-08-14,Bolton,Fulham,0.0,0.0,D,0.0,0.0,D,13.0,12.0,9.0,7.0,12.0,13.0
3,2010-08-14,Chelsea,West Brom,6.0,0.0,H,2.0,0.0,H,18.0,10.0,13.0,4.0,10.0,10.0
4,2010-08-14,Sunderland,Birmingham,2.0,2.0,D,1.0,0.0,H,6.0,13.0,2.0,7.0,13.0,10.0


In [6]:
BM = (df1.Home_Team == 'Man United') 
BM2 =  (df1.Away_Team == 'Man United')
man_u_home_df = df1[BM]
man_u_away_df = df1[BM2]

In [8]:
man_u_home_df

,Game_Date,Home_Team,Away_Team,Full_Time_Home_Team_Goals,Full_Time_Away_Team_Goals,Full_Time_Result,Half_Time_Home_Team_Goals,Half_Time_Away_Team_Goals,Half_Time_Result,Home_Team_Shots,Away_Team_Shots,Home_Team_Shots_on_Target,Away_Team_Shots_on_Target,Home_Fouls,Away_Fouls
9,2010-08-16,Man United,Newcastle,3.0,0.0,H,2.0,0.0,H,18.0,7.0,10.0,3.0,9.0,5.0
23,2010-08-28,Man United,West Ham,3.0,0.0,H,1.0,0.0,H,18.0,7.0,11.0,2.0,6.0,11.0
48,2010-09-19,Man United,Liverpool,3.0,2.0,H,1.0,0.0,H,16.0,7.0,7.0,1.0,15.0,10.0
74,2010-10-16,Man United,West Brom,2.0,2.0,D,2.0,0.0,H,16.0,7.0,9.0,2.0,10.0,7.0
94,2010-10-30,Man United,Tottenham,2.0,0.0,H,1.0,0.0,H,18.0,11.0,10.0,5.0,12.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,2017-01-04,Man United,West Brom,0.0,0.0,D,0.0,0.0,D,18.0,3.0,3.0,1.0,15.0,13.0
295,2017-04-04,Man United,Everton,1.0,1.0,D,0.0,1.0,A,18.0,11.0,3.0,4.0,10.0,18.0
320,2017-04-16,Man United,Chelsea,2.0,0.0,H,1.0,0.0,H,9.0,5.0,3.0,0.0,16.0,20.0
340,2017-04-30,Man United,Swansea,1.0,1.0,D,1.0,0.0,H,12.0,12.0,6.0,4.0,11.0,13.0


In [14]:
# filter by first 3 years
# last home game 12 may 2013
Alex_Man_U = man_u_home_df[(man_u_home_df['Game_Date'] >= '2010-08-16') & (man_u_home_df['Game_Date'] <= '2013-05-12')]
Alex_Man_U

# make full time results and half time results numeric/ binary
# H = win D = draw A = loss
Alex_Man_U['Full_Time_Result'] = Alex_Man_U['Full_Time_Result'].apply(lambda x: 1 if x == "H" else(2 if x =='D' else 3))

Alex_Man_U['Half_Time_Result'] = Alex_Man_U['Half_Time_Result'].apply(lambda x: 1 if x == "H" else(2 if x =='D' else 3))

Alex_Man_U



C:\Users\ekowa\AppData\Local\Temp/ipykernel_24052/2139747477.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Alex_Man_U['Full_Time_Result'] = Alex_Man_U['Full_Time_Result'].apply(lambda x: 1 if x == "H" else(2 if x =='D' else 3))
C:\Users\ekowa\AppData\Local\Temp/ipykernel_24052/2139747477.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Alex_Man_U['Half_Time_Result'] = Alex_Man_U['Half_Time_Result'].apply(lambda x: 1 if x == "H" else(2 if x =='D' else 3))


,Game_Date,Home_Team,Away_Team,Full_Time_Home_Team_Goals,Full_Time_Away_Team_Goals,Full_Time_Result,Half_Time_Home_Team_Goals,Half_Time_Away_Team_Goals,Half_Time_Result,Home_Team_Shots,Away_Team_Shots,Home_Team_Shots_on_Target,Away_Team_Shots_on_Target,Home_Fouls,Away_Fouls
9,2010-08-16,Man United,Newcastle,3.0,0.0,1,2.0,0.0,1,18.0,7.0,10.0,3.0,9.0,5.0
23,2010-08-28,Man United,West Ham,3.0,0.0,1,1.0,0.0,1,18.0,7.0,11.0,2.0,6.0,11.0
48,2010-09-19,Man United,Liverpool,3.0,2.0,1,1.0,0.0,1,16.0,7.0,7.0,1.0,15.0,10.0
74,2010-10-16,Man United,West Brom,2.0,2.0,2,2.0,0.0,1,16.0,7.0,9.0,2.0,10.0,7.0
94,2010-10-30,Man United,Tottenham,2.0,0.0,1,1.0,0.0,1,18.0,11.0,10.0,5.0,12.0,10.0
135,2010-11-20,Man United,Wigan,2.0,0.0,1,1.0,0.0,1,17.0,10.0,12.0,4.0,10.0,14.0
144,2010-11-27,Man United,Blackburn,7.0,1.0,1,3.0,0.0,1,15.0,3.0,11.0,2.0,6.0,6.0
168,2010-12-13,Man United,Arsenal,1.0,0.0,1,1.0,0.0,1,14.0,11.0,10.0,6.0,7.0,16.0
176,2010-12-26,Man United,Sunderland,2.0,0.0,1,1.0,0.0,1,14.0,6.0,10.0,2.0,9.0,10.0
202,2011-04-01,Man United,Stoke,2.0,1.0,1,1.0,0.0,1,18.0,5.0,7.0,1.0,6.0,11.0


In [ ]:
import statsmodels.api as sm



# Add an intercept column to the DataFrame
Alex_Man_U['Intercept'] = 1

# Select the predictors and the target variable
predictors = Alex_Man_U.drop('Full_Time_Result', axis=1)
target = Alex_Man_U['Full_Time_Result'] == 1  # True for win, False otherwise

# Fit the logistic regression model using statsmodels
model = sm.Logit(target, predictors)
result = model.fit()

# Get the p-values for each predictor
p_values = result.pvalues

# Print the p-values
print(p_values)